In [1]:
from utils.data_handling import *
from utils.sensor_optimisation_tsvd import *
from parameters import *
import GPy
import scipy.stats

import matplotlib.pyplot as plt
np.random.seed(101)



### Importing the Tracer Data to Select the Optimisation set S

In [2]:
parameters['i_end'] = 988
parameters['field_name'] = "Tracer"
parameters


{'i_start': 0,
 'i_end': 988,
 'crop': None,
 'field_name': 'Tracer',
 'cov_method': 'sample'}

In [3]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

100%|██████████| 1/1 [00:00<00:00,  6.30it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0


Number of Locations after cropping :  100040
==> Loading from : ../data/temp_data/cache_0_988_None/loc_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/time_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/data_Tracer.pkl


### Working subset of the data : set S

In [4]:
S = working_subset(data_df, loc_df, nbins = (25,25,25), threshold_sum = 10**-2 )

The remaining number of points is :  57725


### Import the Field of interest for computing the optimisation : TracerBackground / Pressure

In [5]:
parameters['i_end'] = 988
parameters['field_name'] = "Tracer"
parameters


{'i_start': 0,
 'i_end': 988,
 'crop': None,
 'field_name': 'Tracer',
 'cov_method': 'sample'}

In [6]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

100%|██████████| 1/1 [00:00<00:00,  6.48it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0


Number of Locations after cropping :  100040
==> Loading from : ../data/temp_data/cache_0_988_None/loc_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/time_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/data_Tracer.pkl


In [7]:
X = loc_df.values[:,:] #[I,:dim]
Z = data_df.values[:,:]#[:,t].reshape(-1,1) # [I,t].reshape(-1,1)
Z.shape

(100040, 989)

### Detrending Data : 
Plotting the data along each direction in order to see if there is a obvious trend. It seams that it is not the case. 

In [8]:
Z[S,:] = (Z[S,:]  - Z[S,:].mean(axis=1,keepdims=True))

### Sensor Optimisation with TSVD : 


Define the Sets for the optimisation

In [9]:
sets = define_sets(S)

Number of sensors to place : 

In [10]:
k = 5

Truncation parameteter for the TSVD : 

In [11]:
tau = 50

In [12]:
A_opt = {}

In [13]:
A_opt['lazy'] = sensor_loc_optimisation_lazy(k,Z, sets, tau)

Inner Loop:   0%|          | 118/57725 [03:29<28:15:42,  1.77s/it]


Inner Loop:   0%|          | 235/57725 [07:00<28:58:32,  1.81s/it]


Inner Loop:   1%|          | 352/57725 [10:30<28:07:30,  1.76s/it]


Inner Loop:   1%|          | 469/57725 [14:00<28:19:09,  1.78s/it]


Inner Loop:   1%|          | 586/57725 [17:32<29:18:38,  1.85s/it]


...

Inner Loop:  11%|█         | 6124/57725 [3:05:56<24:37:58,  1.72s/it]


Inner Loop:  11%|█         | 6236/57725 [3:09:23<25:50:52,  1.81s/it]


Inner Loop:  11%|█         | 6348/57725 [3:12:46<27:07:26,  1.90s/it]


Inner Loop:  11%|█         | 6435/57725 [3:15:28<25:36:44,  1.80s/it]

KeyboardInterrupt: 

In [ ]:
A_opt['naive'] = sensor_loc_optimisation_naive(k,Z, sets, tau)

In [ ]:
A_opt['lazy'] 

In [ ]:
A_opt['naive']